In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import datetime as dt

# Reflect Tables into SQLAlchemy ORM

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

engine = create_engine("sqlite:///hawaii.sqlite")

# reflect an existing database into a new model.creates a base class for an automap schema in SQLAlchemy
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# We can view all of the classes that automap found
Base.classes.keys()

# Save references to each table (class)
Measurement = Base.classes.measurement
Station = Base.classes.station

# Create our session (link) from Python to the DB
session = Session(engine)

In [2]:
# Exploratory Climate Analysis
# Look at Measurements
# Design a query to retrieve the precipitation and temperature data

results = []
results = session.query(Measurement.date, Measurement.prcp, Measurement.tobs).all()


In [3]:
# Save the query results as a Pandas DataFrame and set the index to the date column
df = pd.DataFrame(results, columns=['date','precipitation','tobs'])
df.set_index(df['date'], inplace=True)
df.fillna(0)
# Sort the dataframe by date
df = df.sort_index()
# Use Pandas to calculate the summary statistics for the precipitation data
df.describe()

,precipitation,tobs
count,18103.000000,19550.000000
mean,0.160644,73.097954
std,0.468746,4.523527
min,0.000000,53.000000
25%,0.000000,70.000000
50%,0.010000,73.000000
75%,0.110000,76.000000
max,11.530000,87.000000


In [4]:
df


,date,precipitation,tobs
date,,,
2010-01-01,2010-01-01,0.08,65.0
2010-01-01,2010-01-01,0.21,72.0
2010-01-01,2010-01-01,0.15,75.0
2010-01-01,2010-01-01,0.15,70.0
2010-01-01,2010-01-01,0.05,66.0
...,...,...,...
2017-08-22,2017-08-22,0.50,76.0
2017-08-23,2017-08-23,0.00,81.0
2017-08-23,2017-08-23,0.00,82.0


In [ ]:
# Look at stations
# How many stations are available in this dataset?
session.query(func.count(Station.station)).all()

In [ ]:
# What are the most active stations?
# List the stations and the counts in descending order.
session.query(Measurement.station, func.count(Measurement.station)).\
group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()


In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station
session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
filter(Measurement.station == 'USC00519281').all()

# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station 
results = session.query(Measurement.tobs).\
filter(Measurement.station == 'USC00519281').\
filter(Measurement.date >= prev_year).all()
print(results)

In [ ]:
# convert list of temps to dataframe
temps_df = pd.DataFrame(results, columns=['tobs'])
print(temps_df)

In [ ]:


# and plot the results as a histogram with twelve bins/ranges
temps_df.plot.hist(bins=12)
# compress the x-axis labels so that they fit into the box holding our plot
plt.tight_layout()

In [ ]:
# Perform a query to retrieve the data and precipitation scores for December
results_12 = []
results_12 = session.query(Measurement.date, Measurement.prcp).filter(func.extract('month', Measurement.date) == 12).all()

# Save the query results as a Pandas DataFrame and set the index to the date column
december_rain_df = pd.DataFrame(results_12, columns=['date','precipitation'])
december_rain_df.set_index(december_rain_df['date'], inplace=True)

december_rain_df = december_rain_df.sort_index()
print(december_rain_df.to_string(index=False))


# Use Pandas to calculate the summary statistics for the December precipitation data
december_stats_df = june_rain_df.describe()
december_stats_df

# Use Pandas to calcualte the summary statistics for the December precipitation data
june_stats_df.reset_index(level=0, inplace=True)
december_stats_df.reset_index(level=0, inplace=True)

june_stats_df.fillna(0)
december_stats_df.fillna(0)

statistics_df = pd.merge(june_stats_df, december_stats_df, on='index', suffixes=['_june','_dec'])

statistics_df

statistics_df = statistics_df.rename(columns ={'index':'Statistic'})
statistics_df = statistics_df.rename(columns ={'precipitation_june':'Jun'})
statistics_df = statistics_df.rename(columns ={'precipitation_dec':'Dec'})

statistics_df